In [ ]:
!pip install tensorflow==2.15.0
!pip install keras==2.15.0

In [ ]:
fp = open("kaggle.json", "w")
fp.write("{\"username\":\"skandapr\",\"key\":\"cce87a329f0ee198c5016acac1ff0560\"}")
fp.close()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d sriharshaeedala/financial-fraud-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/sriharshaeedala/financial-fraud-detection-dataset
License(s): CC-BY-SA-4.0
 97% 172M/178M [00:02<00:00, 76.8MB/s]
100% 178M/178M [00:02<00:00, 65.8MB/s]


In [ ]:
!unzip financial-fraud-detection-dataset.zip


Archive:  financial-fraud-detection-dataset.zip
  inflating: Synthetic_Financial_datasets_log.csv  


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("Synthetic_Financial_datasets_log.csv")
df.drop(["nameOrig", "nameDest"], inplace=True, axis=1)
df.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   oldbalanceDest  float64
 6   newbalanceDest  float64
 7   isFraud         int64  
 8   isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 436.9+ MB


In [ ]:
oh_type = pd.get_dummies(df.type)
df.drop(["type"], inplace=True, axis=1)

df = (df - df.min()) / (df.max() - df.min())

df = df.join(oh_type)

df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,0.0,0.000106,0.002855,0.003233,0.000000,0.0,0.0,0.0,False,False,False,True,False
1,0.0,0.000020,0.000357,0.000391,0.000000,0.0,0.0,0.0,False,False,False,True,False
2,0.0,0.000002,0.000003,0.000000,0.000000,0.0,1.0,0.0,False,False,False,False,True
3,0.0,0.000002,0.000003,0.000000,0.000059,0.0,1.0,0.0,False,True,False,False,False
4,0.0,0.000126,0.000697,0.000603,0.000000,0.0,0.0,0.0,False,False,False,True,False


In [ ]:
df["CASH_IN"] = df["CASH_IN"].replace({False: 0.0, True: 1.0})
df["CASH_OUT"] = df["CASH_OUT"].replace({False: 0.0, True: 1.0})
df["DEBIT"] = df["DEBIT"].replace({False: 0.0, True: 1.0})
df["PAYMENT"] = df["PAYMENT"].replace({False: 0.0, True: 1.0})
df["TRANSFER"] = df["TRANSFER"].replace({False: 0.0, True: 1.0})

y = df["isFraud"]
x = df.drop(columns=["isFraud","isFlaggedFraud","step", "oldbalanceDest", "newbalanceDest"])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input

model = Sequential([
    Input(shape=(x.shape[1],)),
    Dense(128, activation="relu"),
    Dense(256, activation="relu"),
    Dense(512, activation="relu"),

    Dense(512, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               1152      
                                                                 
 dense_19 (Dense)            (None, 256)               33024     
                                                                 
 dense_20 (Dense)            (None, 512)               131584    
                                                                 
 dense_21 (Dense)            (None, 512)               262656    
                                                                 
 dense_22 (Dense)            (None, 128)               65664     
                                                                 
 dense_23 (Dense)            (None, 1)                 129       
                                                                 
Total params: 494209 (1.89 MB)
Trainable params: 49420

In [ ]:
epochs = 3
batch_size = 128

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/3
33553/33553 [==============================] - 169s 5ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.0047 - val_accuracy: 0.9992
Epoch 2/3
33553/33553 [==============================] - 159s 5ms/step - loss: 0.0047 - accuracy: 0.9991 - val_loss: 0.0041 - val_accuracy: 0.9992
Epoch 3/3
33553/33553 [==============================] - 168s 5ms/step - loss: 0.0042 - accuracy: 0.9992 - val_loss: 0.0035 - val_accuracy: 0.9992


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

49708/49708 [==============================] - 128s 3ms/step - loss: 0.0036 - accuracy: 0.9992
Test Loss: 0.00357557344250381
Test Accuracy: 0.9992015957832336


In [ ]:
from tensorflow.keras.models import load_model

model.save('Financial_Fraud_Detection_Model.h5')

# To Load the model
#loaded_model = load_model('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd

loaded_model = load_model('Financial_Fraud_Detection_Model.h5')

text = '(92345.0,2546892.0,2454547.0,0.0,0.0,0.0,0.0,1.0)'

test = [eval(text)]

test_pd = pd.DataFrame(test,columns=["amount","oldbalanceOrg","newbalanceOrig","CASH_OUT","CASH_IN","DEBIT","PAYMENT","TRANSFER"])

y_pred = loaded_model.predict(test_pd)


1/1 [==============================] - 0s 89ms/step


array([[1.]], dtype=float32)

In [ ]:
print(y_pred[0][0])

1.0


In [ ]:
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
import sys
print(sys.version)


2.15.0
2.15.0
3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
